## MATH Fewshot COT

In [ ]:
from tqdm import tqdm, trange
import json
import pandas as pd
import time
import openai

openai.api_type = "azure"
openai.api_base = ""
openai.api_version = ""
openai.api_key = ""


def gpt4(query, i, counter=0):
    
    try:
        messages = [
                       {"role": "system", "content": "You are a helpful AI assistant."},
                       {"role": "user", "content": query}
                   ]
        
        response = openai.ChatCompletion.create(
            engine="gpt-4",  # The deployment name you chose when you deployed the ChatGPT or GPT-4 model.
            messages=messages,
            temperature=0
        )
        return response['choices'][0]['message']['content'], response['usage']
    
    except Exception as e:
        
        if counter < 15:
            time.sleep(5)
            return gpt4(prompt, i, counter + 1)
        else:
            print(f"exception {e} at:" + str(i))
            return "", {
    "completion_tokens": 0,
    "prompt_tokens": 0,
    "total_tokens": 0
}
        
        
df = pd.read_csv("./data/MATH.csv")

def get_fewshot_cot_prompt(n):
    System_prompt = """
You are Terence Tao, a distinguished Australian-American mathematician renowned for your substantial contributions to harmonic analysis, partial differential equations, combinatorics, and representation theory. 
Your significant work in multiple areas of mathematics earned you the Fields Medal in 2006, one of the highest honors in the field.

[Examples]
[Problem]:
Let $z$ and $w$ be complex numbers such that $|z| = 1$ and $|w| = 3$. If $|z+w| = 2$, what is $ \\left | \\frac{1}{z} + \\frac{1}{w} \\right|$?

Let's think step by step:
[Solution]:
We simplify the desired expression \\[\n\\left | \\frac{1}{z} + \\frac{1}{w} \\right| = \\left | \\frac{w+z}{wz} \\right|.\n\\]Now, using the fact that $|ab| = |a|\\cdot |b|$ and $|a/b| = |a|/|b|$, we substitute in the values for the magnitudes given in the problem: \\[\n\\left | \\frac{w+z}{wz} \\right| = \\frac{|w+z|}{|w|\\cdot|z|} = \\frac{2}{(1)(3)} = \\boxed{\\frac{2}{3}}.\n\\]

[End of Examples]
"""
        
    return System_prompt + "[Problem]: \n" + df.iloc[n]['problem'] + "\n\nLet's think step by step:\nGive the final answer in \\boxed{}\n[Solution]:"


    
final_json = []
for i in trange(len(df), desc=f"{setting}"):

    prompt = get_fewshot_cot_prompt(i)

    res, usage = gpt4(prompt, i)

    data = df.iloc[i]

    final_json.append({
        "index": i,
        "problem": data['problem'],
        "level": data['level'],
        "type": data['type'],
        "solution": data['solution'],
        "label": data['label'],
        "pred": res,
        "usage": usage
        })

    if i%20==0:
        with open(f"gpt4_{setting}_pred.json", 'w') as f:
            json.dump(final_json, f)


## Scoring

In [1]:
import json
import re

def read_json(path):
    with open(path, 'r') as f:
        s = json.loads(f.read())
    return s

def extract_answer(text):
    
    final_string = ''
    bracket_stack = []
    
    for char in text:
        if char == "{":
            bracket_stack.append(char)
            
        elif char == "}":
            if len(bracket_stack)==0:
                return final_string
            else:
                bracket_stack.pop(-1)
        final_string += char

def convert_fractions(string): #  \frac{23}{45}
    
    def extract_no_from_frac(match):
        l = re.findall(r'{[^}]*}', match)
        #print(l)
        return l[0][1:-1], l[1][1:-1]

    string = string.replace("\frac", " \\frac")
    string = string.replace("\tfrac", " \\frac")
    string = string.replace("\\tfrac", " \\frac")
    Matches = re.findall(r'\\frac{[\S]*}', string)
    
    for match in Matches:
        a, b = extract_no_from_frac(match)
        string = string.replace(match, f"{a}/{b}")

    return string.replace("$", "").replace("\\right)", ")").replace("\left( ", "(")

convert_fractions("\\frac{1}{24}")


def extract_text(text): # \text{neither}
    
    text = text.replace("\text", "\\text")
    #print(text)
    Matches = re.findall(r'\\text{[\S]*}', text)
    #print(Matches)
    for match in Matches:
        f = re.findall(r'{.*?}', match)
        #print(f)
        text = text.replace(match, f[0][1:-1])
    return text


extract_text("\text{neither}")

def process(text):
    
    if not text:
        return text
    text = convert_fractions(text)
    text = extract_text(text)
    
    return text

process("\frac{1}{24} \text{hello}")


def get_accuracy(dataset):
    count = 0
    for i in range(len(dataset)):

        try:
            if process(dataset[i]['label']).strip() == process(extract_answer(dataset[i]['pred'].split("\\boxed{")[-1])).strip():
                count+=1
        except:
            continue
    
    print(f"Total correct {count} out of {len(dataset)}")
    print(f"Accuracy: {round((count/len(dataset))*100, 3)}%")


In [2]:
fewshot_cot = read_json("./output/gpt4_fewshot-CoT_pred.json")
get_accuracy(fewshot_cot)

Total correct 6338 out of 12349
Accuracy: 51.324%
